In [8]:
import cv2
import numpy as np
import pickle
 
# Load your pre-trained model for hand gesture classification
with open('./model_rps.pkl', 'rb') as file:
    model = pickle.load(file)

cap = cv2.VideoCapture(0)
 
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
 
    # Define a region of interest (ROI)
    height, width = frame.shape[:2]
    roi_frame = frame[int(height/2):height, int(width/2):width]
 
    # Convert to grayscale
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
 
    # Apply adaptive Gaussian thresholding to create a binary image
    thresh_roi = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
 
    # Use morphological operations to clean up the thresholded image
    kernel = np.ones((5,5), np.uint8)
    cleaned_roi = cv2.morphologyEx(thresh_roi, cv2.MORPH_CLOSE, kernel, iterations=2)
 
    # Find contours in the cleaned ROI
    contours, _ = cv2.findContours(cleaned_roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]  # Get the largest contour
 
    if contours:
        # Draw the largest contour which is the hand on the ROI
        hand_contour = contours[0]
        cv2.drawContours(roi_frame, [hand_contour], -1, (0, 255, 0), 2)
 
        # Create a mask for the hand in the ROI
        mask = np.zeros_like(gray_roi)
        cv2.drawContours(mask, [hand_contour], -1, (255), thickness=cv2.FILLED)
 
        # Extract the hand from the ROI
        hand_segment = cv2.bitwise_and(roi_frame, roi_frame, mask=mask)
        # Optionally, blur the background and keep the hand sharp
        blurred_background = cv2.GaussianBlur(roi_frame, (15, 15), 0)
        clear_hand = np.where(mask[..., None].astype(bool), roi_frame, blurred_background)
        # Resize the hand segment to your model's expected input size
        hand_segment_resized = cv2.resize(hand_segment, (128, 128))
 
        # Preprocess the hand segment as required by your model before prediction
        hand_segment_normalized = hand_segment_resized / 255.0
 
        # Make prediction
        prediction = model.predict(np.expand_dims(hand_segment_normalized, axis=0))
        # Interpret the prediction and annotate the frame with the result
        predicted_class = np.argmax(prediction)
        cv2.putText(frame, f'Prediction: {predicted_class}', (int(width/2) + 10, int(height/2) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the resulting frame with the ROI highlighted
    cv2.rectangle(frame, (int(width/2), int(height/2)), (width, height), (255, 0, 0), 2)
    cv2.imshow('Hand Segmentation', frame)
 
    # Display the clear hand with blurred background
    cv2.imshow('Hand with Clear Background', clear_hand)
 
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 140ms/step
